In [ ]:
pip install tensorflow


In [ ]:
pip install numpy

In [ ]:
pip install matplotlib


In [ ]:
#Importing all required libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
#Loading the Data SET
url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
zip_path = tf.keras.utils.get_file('cats_and_dogs_filtered.zip', origin=url, extract=True)



In [ ]:
#Providind directory address
base_dir = os.path.join(os.path.dirname('/content/sample_data/cats_and_dogs_filtered_extracted/'), 'cats_and_dogs_filtered')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

print("Train directory:", train_dir)
print("Validation directory:", validation_dir)

In [ ]:
#Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)
print(f"Train samples: {train_generator.n}")
print(f"Validation samples: {validation_generator.n}")


In [ ]:
#Augmented Images Checking
class_names = list(train_generator.class_indices.keys())
images, labels = next(train_generator)

plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(images[i])
    plt.title(f"Label: {class_names[int(labels[i])]}")
    plt.axis("off")
plt.suptitle("Augmented Training Samples")
plt.show()

In [ ]:
#CNN Model Creation


model = models.Sequential([
    # Block 1
    layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(150, 150, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),

    # Block 2
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),

    # Block 3
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),

    # Block 4
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),

    # Global average pooling instead of flatten
    layers.GlobalAveragePooling2D(),

    # Dense layers
    layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

In [ ]:
#Training model
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=15,
    validation_data=validation_generator,
    validation_steps=50
)

In [ ]:
#Model Accuracy Calculation
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Train Accuracy')
plt.plot(epochs_range, val_acc, label='Val Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Train Loss')
plt.plot(epochs_range, val_loss, label='Val Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.show()

In [ ]:
#Accuracy Of the Model
loss, accuracy = model.evaluate(validation_generator, steps=len(validation_generator))
print(f'Validation Loss: {loss:.4f}')
print(f'Validation Accuracy: {accuracy:.4f}')

In [ ]:
#Using Gradio as a interface to interact with our model
#Here we can provide an image of cat or dog to to our model for the classification
import gradio as gr
import numpy as np
from PIL import Image

def predict_image(img):
    img = img.resize((150, 150))
    img = np.array(img) / 255.0
    img = img.reshape(1, 150, 150, 3)

    prediction = model.predict(img)
    predicted_class = "Cat" if prediction < 0.5 else "Dog"
    confidence = 1 - abs(prediction - 0.5)

    return f"Predicted Class: {predicted_class} (Confidence: {confidence[0][0]:.2f})"

interface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Image Classifier",
    description="Upload an image to classify using the trained CNN model."
)

interface.launch(share=True)


Hosting on web via the Huggingface

In [ ]:
#saving the model offline
from keras.saving import save_model

save_model(model, "model.keras")



In [ ]:
#creating app.py
with open("app.py", "w") as f:
    f.write("""
import gradio as gr
import numpy as np
from PIL import Image
from keras.models import load_model

# Load trained Keras model
model = load_model("model.keras")

# Prediction function
def predict_image(img):
    img = img.resize((150, 150))
    img = np.array(img) / 255.0
    img = img.reshape(1, 150, 150, 3)
    prediction = model.predict(img)[0][0]
    label = "Cat" if prediction < 0.5 else "Dog"
    confidence = 1 - abs(prediction - 0.5)
    return f"{label} (Confidence: {confidence:.2f})"

# Gradio Interface
demo = gr.Interface(fn=predict_image,
                    inputs=gr.Image(type="pil"),
                    outputs="text",
                    title="Cat vs Dog Classifier",
                    description="Upload an image to classify it as Cat or Dog")

demo.launch()
""")


In [ ]:
#requirements for the app.py
with open("requirements.txt", "w") as f:
    f.write("gradio\nkeras\ntensorflow\npillow\nnumpy")


In [ ]:
#importing Huggingface to deploy app.py
!pip install -q huggingface_hub

from huggingface_hub import notebook_login
notebook_login()


In [ ]:
#uploading to huggingfase space
from huggingface_hub import create_repo, upload_file
repo_id = "mincater/cat-vs-dog-classifier"
create_repo(repo_id=repo_id, repo_type="space", space_sdk="gradio", exist_ok=True)
upload_file(path_or_fileobj="app.py", path_in_repo="app.py", repo_id=repo_id, repo_type="space")
upload_file(path_or_fileobj="model.keras", path_in_repo="model.keras", repo_id=repo_id, repo_type="space")
upload_file(path_or_fileobj="requirements.txt", path_in_repo="requirements.txt", repo_id=repo_id, repo_type="space")